## 보팅방식 모델 구현 <hr>
- 데이터 : sklearn.datasets의 breast_cancer
- 유형 : 지도학습 + 분류
- 방법 : Voting방식으로 진행 => LogisticRegression, DecisionTreeClassifier, SVC
- 학습데이터셋 : 동일한 데이터셋으로 3개의 모델로 학습 진행

[1] 모듈 로딩 및 데이터 준비 <hr>

In [100]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer

In [101]:
X, y = load_breast_cancer(as_frame=True, return_X_y=True)

In [102]:
X.shape, y.shape

((569, 30), (569,))

In [103]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,
                                                   y,
                                                   random_state=5,
                                                   test_size=0.2,
                                                   stratify=y)

[2] 학습 진행 <hr>

[2-1] 앙상블 보팅 학습에 사용할 모델 인스턴스 생성

In [104]:
# LogisticRegression 인스턴스
from sklearn.linear_model import LogisticRegression

lr_model = LogisticRegression(solver='liblinear')
lr_model.fit(X_train, y_train)

LogisticRegression(solver='liblinear')

[2-2] Ensemble 알고리즘 기반 분류

In [105]:
# DecisionTreeClassifier 인스턴스
from sklearn.tree import DecisionTreeClassifier

dt_model = DecisionTreeClassifier()
dt_model.fit(X_train, y_train)

DecisionTreeClassifier()

[2-3] SVC

In [106]:
# SVC 인스턴스
from sklearn.svm import SVC

svc_model = SVC(probability=True)
svc_model.fit(X_train, y_train)

SVC(probability=True)

In [107]:
from sklearn.ensemble import VotingClassifier

In [108]:
# 동일 데이터셋으로 병렬 학습 진행할 모델 리스트 선정 및 결과 결정 방법 설정
vc_models = VotingClassifier(estimators=[('lr_model', lr_model), 
                                        ('dt_model', dt_model), 
                                        ('svc_model', svc_model)],
                            verbose=True
                            )


In [109]:
# 동일 데이터셋을 전달해서 3개의 모델 동시에 학습 진행
vc_models.fit(X_train, y_train)

[Voting] ................. (1 of 3) Processing lr_model, total=   0.0s
[Voting] ................. (2 of 3) Processing dt_model, total=   0.0s
[Voting] ................ (3 of 3) Processing svc_model, total=   0.0s


VotingClassifier(estimators=[('lr_model',
                              LogisticRegression(solver='liblinear')),
                             ('dt_model', DecisionTreeClassifier()),
                             ('svc_model', SVC(probability=True))],
                 verbose=True)

- 3개 알고리즘 모델에 대한 성능 평가

In [110]:
# 예측하기
new_data = pd.DataFrame([X_test.iloc[0]], columns=X_test.columns)
new_data

#X_test.iloc[0].to_frame().T 같은 결과

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
525,8.571,13.1,54.53,221.3,0.1036,0.07632,0.02565,0.0151,0.1678,0.07126,...,9.473,18.45,63.3,275.6,0.1641,0.2235,0.1754,0.08512,0.2983,0.1049


In [111]:
# 예측하기
vc_models.predict(new_data)

array([1])

In [112]:
# 보팅 인스턴스 내의 학습기들 => 접근 방법 (1)
vc_models.estimators_

[LogisticRegression(solver='liblinear'),
 DecisionTreeClassifier(),
 SVC(probability=True)]

In [113]:
# 보팅 인스턴스 내의 학습기들 => 접근 방법 (2)
vc_models.named_estimators_

{'lr_model': LogisticRegression(solver='liblinear'),
 'dt_model': DecisionTreeClassifier(),
 'svc_model': SVC(probability=True)}

In [114]:
for key, value in vc_models.named_estimators_.items():
    print(key, value.predict(new_data)[0])

lr_model 1
dt_model 1
svc_model 1


In [115]:
print(f'[LogisticRegression]Train : {lr_model.score(X_train, y_train)}, Test : {lr_model.score(X_test, y_test)}')
print(f'[DecisionTreeClassifier]Train : {dt_model.score(X_train, y_train)}, Test : {dt_model.score(X_test, y_test)}')
print(f'[RandomForestClassifier]Train : {vc_models.score(X_train, y_train)}, Test : {vc_models.score(X_test, y_test)}')

[LogisticRegression]Train : 0.9582417582417583, Test : 0.9649122807017544
[DecisionTreeClassifier]Train : 1.0, Test : 0.9210526315789473
[RandomForestClassifier]Train : 0.967032967032967, Test : 0.9385964912280702
